In [ ]:
# creating the path (PYTHONPATH) to our module.
# assuming that our 'pyECM' directory is out ('..') of our current directory 
import os
import sys
module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from pyppm.ssc_cloppa import Cloppa
from pyppm.ssc_pol_prop import Prop_pol
from pyppm.help_functions import extra_functions
from pyscf import scf, gto

Let's going to import a molden file with localized molecular orbitals, in this 
case, for ethane molecule in cc-pvdz basis

In [ ]:
mol, mo_coeff, mo_occ = extra_functions(molden_file='C2H6_ccpvdz_Pipek_Mezey.molden').extraer_coeff

With the mol object, we calculate FC and PSO contribution to total J-coupling between Hydrogen 3 and 7 nuclei

In [ ]:
mf = scf.RHF(mol)
mf.kernel()
ssc_pso_canonical = Prop_pol(mf).ssc(PSO=True, FC=False, atom1='H3', atom2='H7')
ssc_fc_canonical = Prop_pol(mf).ssc(FC=True, atom1='H3', atom2='H7')
print('PSO = ',ssc_pso_canonical)
print('FC = ',ssc_fc_canonical)

Now, we calculate the same with CLOPPA method, for which we must use the LMOs imported previously and the occupied orbitals information.
The variable 'all_pathways = True' indicates that all coupling pathways available in the system is going to be used 
$$
J^X(H_3,H_7) = \sum_{ia} \sum_{jb} J^X_{ia,jb}(H_3,H_7)
$$

In [ ]:
cloppa_obj = Cloppa(mo_coeff_loc=mo_coeff, mol_loc=mol, mo_occ_loc=mo_occ)
ssc_FC_cloppa = cloppa_obj.ssc_pathway(atom1='H3', atom2='H7',  
                            FC=True, PSO=False, FCSD=False, all_pathways=True)
ssc_PSO_cloppa = cloppa_obj.ssc_pathway(atom1='H3', atom2='H7',  
                            FC=False, PSO=True, FCSD=False, all_pathways=True)
print('PSO = ', ssc_PSO_cloppa)
print('FC = ', ssc_FC_cloppa)

We could fix two occupied LMOs  summing over all virtual LMOs, in order to know their contribution to a $J^X$.
$$
J^X_{i,j}(H_3,H_7) = \sum_{ab}J^X_{ia,jb}(H_3,H_7)
$$

In this example, we are going to studied the coupling pathway conform by contribution of bonding and antibonding to Fermi-Contact mechanism

In [ ]:
ssc_FC_path_ij = cloppa_obj.ssc_pathway(atom1='H3', atom2='H7', occ_atom1=3, 
                            occ_atom2=5, FC=True ,PSO=False, FCSD=False)
print(ssc_FC_path_ij)


<img src="occ_ethane.png" width="700"> 
<div>

This are the occupied LMOs chosen and they have a contribution of 8.864 Hz to total $J(H_3,H_7)$

Then, as the occupied orbitals 3 and 5 has big contribution to J, we are interested in know which are the virtual LMOs that contributes to that pathway $J^{FC}_{ia,jb}(H_3,H_7)$ . For that, we choose, for example, 37 and 38 virtual LMOs
<div>
<img src="anti_1s_etano.png" width="700">

In [ ]:
ssc_FC_path_iajb = cloppa_obj.ssc_pathway(atom1='H3', atom2='H7', 
                                     occ_atom1=3, vir_atom1=36,  
                                     occ_atom2=5, vir_atom2=37,
                                     FC=True)
print(ssc_FC_path_iajb) 

Which tell us that those LMOs has a contribution of 11.4 Hz 

Finally, we can also evaluate the perturbators $\textbf{b}$ and principal propagator $\textbf{P}$ element that conform that response
$$
J^{FC}_{ia,jb} = b^{FC}_{ia}(H_3) * {^3}P_{ia,jb} * b^{FC}_{jb}(H_7)
$$

In [ ]:
p1, m, p2 = cloppa_obj.pathway_elements(FC=False, FCSD=True, PSO=False,
                        atom1='H3', occ_atom1=3, vir_atom1=36,
                        atom2='H7', occ_atom2=5, vir_atom2=37)

print('b^{FC}_{ia}(H_3) =',p1)
print('P_{ia,jb} =', m)
print('b^{FC}_{jb}(H_7) =',p2)


